In [14]:
"""Main module."""
import ipyleaflet 

class Map(ipyleaflet.Map):

    def __init__(self,**kwargs):

        super().__init__(**kwargs):

SyntaxError: invalid syntax (<ipython-input-14-60ca505a4a09>, line 8)